In [1]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from gudhi.representations import Landscape, PersistenceImage
from gudhi.weighted_rips_complex import WeightedRipsComplex
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
import pandas as pd 
import numpy as np
import gudhi
import math
import dcor 

In [2]:
#READ RNA COUNT MATRICES (.TSV) FILES AND MERGE INTO A SINGLE DATASET 

In [3]:
def read_pickle_files(path, path_differential_genes):
    
    pickle_file_directory = '/Users/lebohangmashatola/Library/CloudStorage/Box-Box/Lebo_Mashatola/Scripts/data/rna_data/' + path + '.pkl'
    
    df = pd.read_pickle(pickle_file_directory)
    differential_genes = pd.read_csv(filepath_differentially_expressed_genes, index_col=0) 
    
    differential_genes = differential_genes["x"].to_numpy()    
    gene_exprs_matrix = df[np.intersect1d(df.columns, differential_genes)] #Subset differentially expressed genes
        
    return gene_exprs_matrix
    

In [4]:
LUAD = read_pickle_files('LUAD', 'LUAD_READ.csv')

In [5]:
READ = read_pickle_files('READ', 'LUAD_READ.csv')

In [6]:
#MERGE THE DATASETS (DIFFERENT CANCER TYPES AND PERFORM DATA SPLIT TRAIN (70%) AND TEST (30%)

In [7]:
def merge_split(file1, file2):
    
    labelencoding = preprocessing.LabelEncoder()
    
    gene_exprs_matrix = pd.concat([file1, file2], axis=0)            
    train, test = train_test_split(gene_exprs_matrix, test_size=0.3, train_size=0.7, random_state=0, shuffle=True)
            
    train_labs = labelencoding.fit_transform(train.index.to_list())
    test_labs = labelencoding.fit_transform(test.index.to_list())
    
    return np.array(train), np.array(test), train_labs, test_labs

In [9]:
train_READ_LUAD, test_READ_LUAD, train_labs_READ_LUAD, test_labs_READ_LUAD = merge_split(LUAD, READ)                                                                                                                                                                                                                                          

In [10]:
#INTERGENE CORRELATION MEASURE USING DCOR 

In [11]:
def intergene_correlation_measure(DF):
    
    num_genes = DF.shape[1]
    
    dist = np.zeros((num_genes, num_genes))
    
    for i in range(num_genes):
        for j in range(i+1, num_genes):
            
            dist[i,j] = dcor.distance_correlation(DF[:,i], DF[:,j])
    
    dist = dist + dist.T + np.eye(num_genes)
    
    return dist 

In [12]:
def unit_test_intergene_correlation_measure():
    
    expression = np.array([[22,25,30],[32,23,24], [23, 21, 28]])
    
    return intergene_correlation_measure(expression)   

In [13]:
Z_READ_LUAD = intergene_correlation_measure(train_READ_LUAD)

In [81]:
#DISTANCE CORRELATION MEASURE 

In [14]:
M_READ_LUAD = 1 - Z_READ_LUAD

In [15]:
#PER-PATIENT DISTANCE MATRIX ADDING FROM GLOBAL DISTANCE MATRIX

In [16]:
def patient_correlation_measure(F, M):
    
    dist = np.zeros((M.shape[1], M.shape[1]))
    num_genes = M.shape[1]
    
    for i in range(num_genes):
        
        for j in range(i+1, num_genes):
            
            b = abs((F[i]**2 - F[j]**2) / M[i,j])
            dist[i,j] = math.sqrt(M[i, j]**2 + b**2 + 2*F[i]**2 + 2*F[j]**2)/2 #weighting algorithm derived from Mandal et al., 2020
    
    dist = dist + dist.T + np.eye(num_genes)
    
    return dist

In [17]:
def unit_test_patient_correlation_measure():
    
    X = [1,2,3]
    M = np.array([[1, 0.3, 0.02],[0.3, 1, 0.2], [0.02, 0.2, 1]])
    
    return patient_correlation_measure(X, M)

In [18]:
#TOPOLOGICAL DATA ANALYSIS USING GUDHI, RETURNS PER-PATIENT PERSISTENT DIAGRAMS IN ZERO-TH DIMENSION 

In [89]:
def simplicial_patient(X, M, name):
    
    Persistent_diagrams0, Persistent_diagrams1 = [], []
    
    for s in X:
        
        distance_matrix = patient_correlation_measure(s, M)
        
        rips_complex = RipsComplex(distance_matrix).create_simplex_tree(max_dimension=1) #Weights used include per-patient gene expressions
        
        rips_complex.collapse_edges()
        rips_complex.expansion(2)
        rips_complex.persistence()
    
        Persistent_diagrams0.append(rips_complex.persistence_intervals_in_dimension(0))
        Persistent_diagrams1.append(rips_complex.persistence_intervals_in_dimension(1))
    
    import matplotlib.pyplot as plt
    filepath = '/home/lmashatola/results/' + name
    gudhi.plot_persistence_diagram(rips_complex.persistence(), legend=True)
    plt.savefig(filepath)
    
    remove_infinity = lambda barcode : np.array([bars for bars in barcode if bars[1]!= np.inf])
    Persistent_diagrams0 = list(map(remove_infinity, Persistent_diagrams0))
    #Persistent_diagrams1 = list(map(remove_infinity, Persistent_diagrams1))
    
    return Persistent_diagrams0   

In [90]:
dgms_READ_LUAD = simplicial_patient(train_READ_LUAD, M_READ_LUAD, 'READ_LUAD.pdf')
dgms_READ_BRCA = simplicial_patient(train_READ_BRCA, M_READ_BRCA, 'READ_BRCA.pdf')
dgms_LUAD_BRCA = simplicial_patient(train_LUAD_BRCA, M_LUAD_BRCA, 'LUAD_BRCA.pdf')

In [91]:
Tdgms_READ_LUAD = simplicial_patient(test_READ_LUAD, M_READ_LUAD, 'Test_READ_LUAD.pdf')
Tdgms_READ_BRCA = simplicial_patient(test_READ_BRCA, M_READ_BRCA, 'Test_READ_BRCA.pdf')
Tdgms_LUAD_BRCA = simplicial_patient(test_LUAD_BRCA, M_LUAD_BRCA, 'Test_LUAD_BRCA.pdf')

In [92]:
#SKLEARN PIPELINE WITH GUDHI PERSISTENT DIAGRAM OUTPUT 

In [93]:
def TDA_representations(train_dataset):
    
    pipe_Landscape = Pipeline([("TDA", Landscape(num_landscapes=10, resolution=len(train_dataset), sample_range=[0,len(train_dataset)])),
                               ("Estimator", MLPClassifier(early_stopping=True))])

    pipe_Image = Pipeline([("TDA", PersistenceImage(resolution=[len(train_dataset), len(train_dataset)], im_range=[10, 10, 10, 10])),
                           ("Estimator", MLPClassifier(early_stopping=True))])
    
    return pipe_Landscape, pipe_Image

In [94]:
pipe_Landscape_READ_LUAD, pipe_Image_READ_LUAD = TDA_representations(dgms_READ_LUAD)
pipe_Landscape_READ_BRCA, pipe_Image_READ_BRCA = TDA_representations(dgms_READ_BRCA)
pipe_Landscape_LUAD_BRCA, pipe_Image_LUAD_BRCA   = TDA_representations(dgms_LUAD_BRCA)

In [95]:
Testing_pipe_Landscape_READ_LUAD, Testing_pipe_Image_READ_LUAD = TDA_representations(Tdgms_READ_LUAD)
Testing_pipe_Landscape_READ_BRCA, Testing_pipe_Image_READ_BRCA = TDA_representations(Tdgms_READ_BRCA)
Testing_pipe_Landscape_LUAD_BRCA, Testing_pipe_Image_LUAD_BRCA = TDA_representations(Tdgms_LUAD_BRCA)

In [96]:
#MODEL TRAINING 

In [97]:
def model_train(dgms, labs, pipe_Landscape, pipe_Image):
        
    trained_model_landscape = pipe_Landscape.fit(dgms, labs)
    trained_model_Image = pipe_Image.fit(dgms, labs)
    
    predicted_label_lanscape = trained_model_landscape.predict(dgms)
    predicted_label_Image = trained_model_Image.predict(dgms)
        
    f1_micro_Landscape = f1_score(labs, predicted_label_lanscape, average="micro")
    f1_macro_Landscape = f1_score(labs, predicted_label_lanscape, average="macro")
    recall_Landscape = recall_score(labs, predicted_label_lanscape)
    precision_Landscape = precision_score(labs, predicted_label_lanscape)
    
    f1_micro_Image = f1_score(labs, predicted_label_Image, average="micro")
    f1_macro_Image = f1_score(labs, predicted_label_Image, average="macro")
    recall_Image = recall_score(labs, predicted_label_Image)
    precision_Image = precision_score(labs, predicted_label_Image)
    
    
    a = np.array([[trained_model_landscape.score(dgms, labs), trained_model_Image.score(dgms, labs)]])
    a = pd.DataFrame(a, index=['Train Accuracy'], columns=['Accuracy Landscape',  'Accuracy Image'])
    a.to_csv('/home/lmashatola/results/training_accuracy.csv')
    
    b = np.array([[f1_micro_Image, f1_macro_Image, precision_Image, recall_Image], [f1_micro_Landscape, f1_macro_Landscape, precision_Landscape, recall_Landscape]])
    b = pd.DataFrame(b, index=['Image', 'Landscape'], columns=['F1 Score Micro', 'F1 Score Macro', 'Precision', 'TPR'])
    b.to_csv('/home/lmashatola/results/training_classification.csv')
    
    return a, b

In [98]:
accuracy_READ_LUAD, classification_metrics_READ_LUAD = model_train(dgms_READ_LUAD, train_labs_READ_LUAD, pipe_Landscape_READ_LUAD, pipe_Image_READ_LUAD) #READ vs LUAD

In [101]:
accuracy_READ_BRCA, classification_metrics_READ_BRCA = model_train(dgms_READ_BRCA, train_labs_READ_BRCA, pipe_Landscape_READ_BRCA, pipe_Image_READ_BRCA) #READ vs BRCA

In [104]:
accuracy_LUAD_BRCA, classification_metrics_LUAD_BRCA = model_train(dgms_LUAD_BRCA, train_labs_LUAD_BRCA, pipe_Landscape_LUAD_BRCA, pipe_Image_LUAD_BRCA) #LUAD vs BRCA

In [107]:
#MODEL TESTING 

In [108]:
def model_test(dgms, labs, Tdgsm, Tlabs, pipe_Landscape, pipe_Image):
        
    trained_model_landscape = pipe_Landscape.fit(dgms, labs)
    trained_model_Image = pipe_Image.fit(dgms, labs)
    
    predicted_label_lanscape = trained_model_landscape.predict(Tdgsm)
    predicted_label_Image = trained_model_Image.predict(Tdgsm)
        
    f1_micro_Landscape = f1_score(Tlabs, predicted_label_lanscape, average="micro")
    f1_macro_Landscape = f1_score(Tlabs, predicted_label_lanscape, average="macro")
    recall_Landscape = recall_score(Tlabs, predicted_label_lanscape)
    precision_Landscape = precision_score(Tlabs, predicted_label_lanscape)
    
    f1_micro_Image = f1_score(Tlabs, predicted_label_Image, average="micro")
    f1_macro_Image = f1_score(Tlabs, predicted_label_Image, average="macro")
    recall_Image = recall_score(Tlabs, predicted_label_Image)
    precision_Image = precision_score(Tlabs, predicted_label_Image)
    
    a = np.array([[trained_model_landscape.score(Tdgsm, Tlabs), trained_model_Image.score(Tdgsm, Tlabs)]])
    a = pd.DataFrame(a, index=['Test Accuracy'], columns=['Accuracy Landscape', 'Accuracy Image'])
    a.to_csv('/home/lmashatola/results/testing_accuracy.csv')
    
    b = np.array([[f1_micro_Image, f1_macro_Image, precision_Image, recall_Image], [f1_micro_Landscape, f1_macro_Landscape, precision_Landscape, recall_Landscape]]])
    b = pd.DataFrame(b, index=['Image', 'Landscape'], columns=['F1 Score Micro', 'F1 Score Macro', 'Precision', 'TPR'])
    b.to_csv('/home/lmashatola/results/testing_classification.csv')
    
    return a, b

In [109]:
Test_accuracy_READ_LUAD, Test_classification_metrics_READ_LUAD = model_test(dgms_READ_LUAD, train_labs_READ_LUAD, Tdgms_READ_LUAD, test_labs_READ_LUAD, Testing_pipe_Landscape_READ_LUAD, Testing_pipe_Image_READ_LUAD) 
#READ vs LUAD

In [ ]:
Test_accuracy_READ_BRCA, Test_classification_metrics_READ_BRCA = model_test(dgms_READ_BRCA, train_labs_READ_BRCA, Tdgms_READ_BRCA, test_labs_READ_BRCA, Testing_pipe_Landscape_READ_BRCA, Testing_pipe_Image_READ_BRCA) 
#READ_BRCA

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
Test_accuracy_LUAD_BRCA, Test_classification_metrics_LUAD_BRCA = model_test(dgms_LUAD_BRCA, train_labs_LUAD_BRCA, Tdgms_LUAD_BRCA, test_labs_LUAD_BRCA,Testing_pipe_Landscape_LUAD_BRCA, Testing_pipe_Image_LUAD_BRCA) 
#LUAD_BRCA